In [9]:
import pandas as pd
print("Hello Portfolio Analysis")

Hello Portfolio Analysis


#

# 1. Read transactions history

In [3]:
file_transactions = 'transactions.csv'
df_trans = pd.read_csv(file_transactions)

In [7]:
df_trans.head()


,Date,Ticker,Action,Quantity,Price,Total
0,2022-01-13,ASML,Sell,20,730,14600.0
1,2022-01-13,AAL,19,100,1900,NaN


In [8]:
df_trans.tail()

,Date,Ticker,Action,Quantity,Price,Total
0,2022-01-13,ASML,Sell,20,730,14600.0
1,2022-01-13,AAL,19,100,1900,NaN
